**Table of contents**<a id='toc0_'></a>    
- [移動ロボットによる強化学習を用いたライントレース](#toc1_)    
- [参考サイトなど](#toc2_)    
- [pybulletの起動](#toc3_)    
- [初期設定](#toc4_)    
- [Q学習クラスの定義](#toc5_)    
  - [Q学習とは](#toc5_1_)    
  - [Q値と報酬](#toc5_2_)    
  - [Q値の更新](#toc5_3_)    
  - [Qテーブル](#toc5_4_)    
- [Qテーブルの初期化](#toc6_)    
- [Q学習のパラメータ設定](#toc7_)    
- [Q学習の実行](#toc8_)    
- [おまけ：ライントレース用の画像生成](#toc9_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[移動ロボットによる強化学習を用いたライントレース](#toc0_)

本notebookでは2輪の移動ロボットを用いて、強化学習を用いたライントレースを行います。

（pybulletで使用可能な関数がまとめられたマニュアルについては[こちら](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf)を参照してください。）


# <a id='toc2_'></a>[参考サイトなど](#toc0_)
- [【強化学習】TD学習](https://www.tcom242242.net/entry/ai-2/%E5%BC%B7%E5%8C%96%E5%AD%A6%E7%BF%92/%E3%80%90%E5%BC%B7%E5%8C%96%E5%AD%A6%E7%BF%92%E3%80%91td%E5%AD%A6%E7%BF%92%EF%BC%88td0/)
- [ゼロからDeepまで学ぶ強化学習](https://qiita.com/icoxfog417/items/242439ecd1a477ece312)
- [強化学習手法の一つ「Q学習」をなるべくわかりやすく解説してみた](https://www.cresco.co.jp/blog/entry/entry-1676566754114828351.html)
- [Q学習の式を理解する](https://note.com/pumonmon/n/n04f9139ad826#0V6ch)
- [【入門】Q学習の解説とpythonでの実装 〜シンプルな迷路問題を例に〜](https://www.tcom242242.net/entry/ai-2/%E5%BC%B7%E5%8C%96%E5%AD%A6%E7%BF%92/%E3%80%90%E5%BC%B7%E5%8C%96%E5%AD%A6%E7%BF%92%E3%80%81%E5%85%A5%E9%96%80%E3%80%91q%E5%AD%A6%E7%BF%92_%E8%BF%B7%E8%B7%AF%E3%82%92%E4%BE%8B%E3%81%AB/#google_vignette)


# <a id='toc3_'></a>[pybulletの起動](#toc0_)

pybulletを起動します。

In [1]:
import time
import math
import pybullet
import pybullet_data
physics_client = pybullet.connect(pybullet.GUI) 

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


# <a id='toc4_'></a>[初期設定](#toc0_)

床の生成、ボックスオブジェクトの生成、ロボットの生成、カメラ位置の設定などの初期設定を行います。


In [2]:
pybullet.resetSimulation() # シミュレーション空間をリセット
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # pybulletに必要なデータへのパスを追加
pybullet.setGravity(0.0, 0.0, -9.8) # 地球上における重力に設定
time_step = 1./240.
pybullet.setTimeStep(time_step)

#床の読み込み
plane_id = pybullet.loadURDF("plane.urdf",  basePosition=[-5.0, -5.0, 0.0], globalScaling=5.0)
tex_uid = pybullet.loadTexture("../texture/line_trace_ground.png")
pybullet.changeVisualShape(plane_id, -1, textureUniqueId=tex_uid)

# ロボットの読み込み
car_start_pos = [0.0, 0.0, 0.1]  # 初期位置(x,y,z)を設定
car_start_orientation = pybullet.getQuaternionFromEuler([0, 0, 0])  # 初期姿勢(roll, pitch, yaw)を設定
car_id = pybullet.loadURDF("../urdf/simple_two_wheel_car.urdf",car_start_pos, car_start_orientation)

# GUIモードの際のカメラの位置などを設定
camera_distance = 6.0
camera_yaw = 180.0 # deg
camera_pitch = -90.1 # deg
# cameraTargetPosition = [4.0, 0.0, 0.0]
# camera_target_position = [2.5, 0.0, 0.0]
camera_target_position = [0.0, 1.0, 0.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

ven = Mesa
ven = Mesa


In [3]:
# 移動ロボットの動作に合わせて、cameraUpVectorを計算するために使用する回転行列を定義
def Rx(theta):
    return np.array([[1, 0, 0],
                     [0, np.cos(theta), -np.sin(theta)],
                     [0, np.sin(theta), np.cos(theta)]])

def Ry(theta):
    return np.array([[np.cos(theta), 0, np.sin(theta)],
                     [0, 1, 0],
                     [-np.sin(theta), 0, np.cos(theta)]])

def Rz(theta):
    return np.array([[np.cos(theta), -np.sin(theta), 0],
                     [np.sin(theta), np.cos(theta), 0],
                     [0, 0, 1]])

# <a id='toc5_'></a>[Q学習クラスの定義](#toc0_)

## <a id='toc5_1_'></a>[Q学習とは](#toc0_)

Q学習とは、強化学習の一種で

ロボットが「状態$s$」を取った時に、「Q値 $Q(s, a)$」を最大化するような「行動$a$」を選択するように学習する手法です。

今回は、下図のように

- 移動ロボットのカメラから得られた「左側の色」と「右側の色」を状態$s_t$
- ロボットの走行方向（直進、右折、左折）を行動$a_t$

として、Q学習を行います。

<img src="../images/MobileRobot/mobile_robot_reinforcement_q_learning/sim_environment.png" width="80%">

## <a id='toc5_2_'></a>[Q値と報酬](#toc0_)

Q学習において、以下の2つの概念が重要です。

- 「Q値 $Q(s, a)$」
- 「報酬$r$」

これらは、似ているようで異なる概念です。

「報酬$r$」は、状態$s$において行動$a$を取った後に得られる報酬を表します。
今回の場合、
- 状態$s_t$において行動$a_t$を取った後に、「左の色が黒」「右の色が黒」となった場合に報酬$r_{t+1}=+1$を与え、
- 「左の色が白」「右の色が白」となった場合に報酬$r_{t+1}=-1$を与え、
- それ以外の場合は、報酬を与えない

ように設定しています（即時報酬とも呼ばれます）。

<img src="../images/MobileRobot/mobile_robot_reinforcement_q_learning/reward.png" width="100%">

一方で、「Q値 $Q(s, a)$」は、**将来の報酬を見越して**、状態$s$において行動$a$を取った場合の価値を表します。


---

---

tips: 報酬とQ値の違いの具体例

報酬とQ値を具体例で比較してみます。
今回の例だと少し分かりにくいので、「囲碁」の例を用いて説明します。
- 囲碁の場合、報酬は「勝敗」です。勝った場合に報酬$r=+1$、負けた場合に報酬$r=-1$を与えます。
- 一方で、Q値は「局面$s$において、次の手$a$を打った場合に、最終的に勝つ確率」を表します

2015年に、DeepMindが開発した「AlphaGo」は、Q値を用いて囲碁の強さを学習し、世界チャンピオンを破ることに成功しました。
これは、「一見その瞬間では不利に見えるようでも、将来的に有利になる手を打つ（=**将来の報酬（勝利）を最大化するような行動を取る**）」という戦略を（人間のプロ棋士を超えるほどの）高い精度で学習した結果です。

なお、AlphaGoの場合は、Deep Q-Network（DQN）という、Q学習にニューラルネットワークを組み合わせた手法を用いています。

---

---

## <a id='toc5_3_'></a>[Q値の更新](#toc0_)

Q学習では、以下の式でQ値を更新することを繰り返すことで学習を行います。

$$
Q(s_t, a_t) = Q(s_t, a_t) + \alpha \left( r_{t+1} + \gamma \max_{a} Q(s_{t+1}, a) - Q(s_t, a_t) \right)
$$

ここで、各変数の意味は以下の通りです。
- $\alpha$：学習率
- $r_{t+1}$：状態$s_t$において、行動$a_t$を取った直後に得られる報酬
- $\gamma$：割引率（未来の報酬をどれだけ重視するか）
- $\max_{a} Q(s_{t+1}, a)$：状態$s_{t+1}$において、取りうる行動$a$の中で最もQ値が高いもの

<br>


　なお、$r_{t+1} + \gamma \max_{a} Q(s_{t+1}, a) - Q(s_t, a_t)$の部分は、「TD誤差」と呼ばれ、  
TD誤差を$\delta = r_{t+1} + \gamma \max_{a} Q(s_{t+1}, a) - Q(s_t, a_t)$として以下のように定義することもあります（コード内でもこの定義を使用しています）。

$$
Q(s_t, a_t) = Q(s_t, a_t) + \alpha \delta
$$

<img src="../images/MobileRobot/mobile_robot_reinforcement_q_learning/td_error.png" width="60%">

<br>
<br>

特に、$r_{t+1} + \gamma \max_{a} Q(s_{t+1}, a)$ の部分を「TDターゲット」と呼び、$Q(s_t, a_t)$がこのTDターゲットに近づくようにQ値を更新していきます。

<img src="../images/MobileRobot/mobile_robot_reinforcement_q_learning/td_target.png" width="60%">

<!-- ---

---

tips:

他の強化学習手法に、SARSA（State-Action-Reward-State-Action）という手法があります。

$$
Q(s_t, a_t) = Q(s_t, a_t) + \alpha \left( r_{t+1} + \gamma Q(s_{t+1}, a_{t+1}) - Q(s_t, a_t) \right)
$$

<img src="../images/MobileRobot/mobile_robot_reinforcement_q_learning/q_learn_and_sarsa.png" width="60%">

Q学習が「最もQ値が高い行動を選択する」のに対して、SARSAは「実際に取った行動に対するQ値を更新する」という違いがあります。

---

--- -->

## <a id='toc5_4_'></a>[Qテーブル](#toc0_)

ここまでで、Q値について説明しましたが、Q値は各状態$s$、行動$a$のペアごとに存在するため、各Q値は下記のような「Qテーブル」と呼ばれるテーブルで管理されます。
 
| states \ actions | $a_0$ | $a_1$ |...| $a_n$ |
|:-----------------:|:-----:|:-----:|:-:|:-----:|
| $s_0$             | $Q(s_0, a_0)$ | $Q(s_0, a_1)$ |...| $Q(s_0, a_n)$ |
| $s_1$             | $Q(s_1, a_0)$ | $Q(s_1, a_1)$ |...| $Q(s_1, a_n)$ |
| ...               | ... | ... |...| ... |
| $s_m$             | $Q(s_m, a_0)$ | $Q(s_m, a_1)$ |...| $Q(s_m, a_n)$ |



今回のライントレース問題におけるQテーブルは下図のようになります。

<img src="../images/MobileRobot/mobile_robot_reinforcement_q_learning/q_table.png" width="100%">


今回のライントレース問題の場合、
- 「黒黒」の場合は直進
- 「白黒」の場合は右折
- 「黒白」の場合は左折

という風に、走行する必要があるので、学習後のQテーブルは下図のようになれば良さそうです（Highと書いてある部分が、学習後にQ値が高くなるべき部分です）。

<img src="../images/MobileRobot/mobile_robot_reinforcement_q_learning/q_table_after_learning.png" width="60%">



※ 今回は勉強用に、学習後のQテーブルを予想できるような簡単な問題設定にしています。  
実際は、複雑な問題になるとQテーブルを予想することは困難なので、適切に報酬などを設定することが重要です。

In [4]:
from enum import IntEnum
import numpy as np
import cv2

class Actions(IntEnum):
    """
    行動の定義（Q-tableの列）
    """
    STRAIGHT = 0
    TURN_LEFT = 1
    TURN_RIGHT = 2

class States(IntEnum):
    """
    状態の定義（Q-tableの行）
    """
    BLACK_BLACK = 0
    WHITE_BLACK = 1
    BLACK_WHITE = 2
    WHITE_WHITE = 3


class QLearning:
    def __init__(self, q_table, LEARNING_RATE=0.1, DISCOUNT_RATE=0.9, EPSILON=0.1):
        """
        Q学習の初期化

        Parameters
        ----------
        q_table : list
            Qテーブル
        LEARNING_RATE : float
            学習率
        DISCOUNT_RATE : float
            割引率
        EPSILON : float
            ランダム行動を選択する確率
        """ 
        self.q_table = q_table
        self.LEARNING_RATE = LEARNING_RATE
        self.DISCOUNT_RATE = DISCOUNT_RATE
        self.EPSILON = EPSILON   

    def get_state(self, car_id, CAMERA_IDX, CAMERA_TARGET_IDX, projection_matrix):
        """
        状態の取得

        Parameters
        ----------
        car_id : int
            ロボットのID
        CAMERA_IDX : int
            カメラリンクのインデックス
        CAMERA_TARGET_IDX : int
            注視点用の仮想的なリンクのインデックス
        projection_matrix : list
            プロジェクション行列

        Returns
        -------
        state : int
            状態 s_t
        """
        # camera_up_vectorのデフォルトの方向
        camera_up_vector = np.array([0, -1, 0])
        
        # カメラリンクの位置を取得
        camera_link_pose = pybullet.getLinkState(car_id, CAMERA_IDX)[0]

        # 注視点用の仮想的なリンクの位置を取得
        camera_target_link_pose = pybullet.getLinkState(car_id, CAMERA_TARGET_IDX)[0] 

        # cameraUpVectorを移動ロボットの姿勢に合わせて回転
        mobile_robot_roll, mobile_robot_pitch, mobile_robot_yaw = pybullet.getEulerFromQuaternion(pybullet.getLinkState(car_id, CAMERA_IDX)[1])
        R = Rz(np.deg2rad(90.0) + mobile_robot_yaw)@Ry(mobile_robot_pitch)@Rx(mobile_robot_roll)
        rotate_camera_up_vector = R@camera_up_vector

        # カメラリンク -> 注視点用の仮想的なリンク 方向のviewMatrixを取得
        view_matrix = pybullet.computeViewMatrix(cameraEyePosition=[camera_link_pose[0], camera_link_pose[1], camera_link_pose[2]],cameraTargetPosition=[camera_target_link_pose[0], camera_target_link_pose[1], camera_target_link_pose[2]],cameraUpVector=[rotate_camera_up_vector[0], rotate_camera_up_vector[1], rotate_camera_up_vector[2]])
        
        # ラインの画像を取得
        width, height, rgb_img, _, _ = pybullet.getCameraImage(600, 300, view_matrix, projection_matrix, renderer=pybullet.ER_BULLET_HARDWARE_OPENGL, shadow=0, flags=pybullet.ER_NO_SEGMENTATION_MASK)

        # 画像の2値化
        img = np.reshape(rgb_img, (height, width, 4))  # 取得した画像を4チャンネルのnumpy配列に変換
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # グレースケールに変換
        _, binary_img = cv2.threshold(gray_img, 127, 255, cv2.THRESH_BINARY)  # 2値化

        # 画像の中心から少し左の位置にあるピクセルの色を取得
        color_left = binary_img[height//2, width//2 - 150]

        # 画像の中心から少し右の位置にあるピクセルの色を取得
        color_right = binary_img[height//2, width//2 + 150]

        # 状態の取得
        if color_left == 0 and color_right == 0:
            state = States.BLACK_BLACK
        elif color_left == 255 and color_right == 0:
            state = States.WHITE_BLACK
        elif color_left == 0 and color_right == 255:
            state = States.BLACK_WHITE
        elif color_left == 255 and color_right == 255:
            state = States.WHITE_WHITE
        return state

    def get_action(self, state):
        """
        行動の取得

        Parameters
        ----------
        state : int
            状態 s_t

        Returns
        -------
        action : int
            行動 a_t
        """

        # ε-greedy法による行動選択
        if np.random.uniform(0, 1) < self.EPSILON:
            # epsilonの確率でランダムに行動を選択
            action = np.random.choice(list(Actions))
        else:
            # 1-epsilonの確率でQ値が最大となる行動を選択
            action = Actions(np.argmax(self.q_table[state][:]))
        return action
    
    def get_reward(self, state):
        """
        報酬の取得

        Parameters
        ----------
        state : int
            状態 s_t

        Returns
        -------
        reward : float
            報酬 r_t
        """
        if state == States.BLACK_BLACK:
            reward = 1
        elif state == States.BLACK_WHITE:
            reward = 0
        elif state == States.WHITE_BLACK:
            reward = 0
        elif state == States.WHITE_WHITE:
            reward = -1
        return reward
    
    def update_q_table(self, state, action, state_next, reward):
        """
        Qテーブルの更新

        Parameters
        ----------
        state : int
            時刻tの状態 s_t
        action : int
            時刻tの行動 a_t
        state_next : int
            時刻t+1の状態 s_{t+1}
        reward : float
            時刻t+1の報酬 r_{t+1}
            ※「時刻tの行動a_t」を取った結果、「時刻tの状態s{t}」→ 「時刻t+1の状態s_{t+1}」に遷移した結果得られる報酬なので、添え字はt+1となる
        """
        # 「時刻t+1の状態s_{t+1}」で可能な「行動a」の中で最大のQ値を求める
        max_q_value = max(self.q_table[state_next][:])

        # TD誤差の計算
        td_error = reward + self.DISCOUNT_RATE * max_q_value - self.q_table[state][action]
                                                            
        # Qテーブルの更新
        self.q_table[state][action] = self.q_table[state][action] + self.LEARNING_RATE * td_error
        

# <a id='toc6_'></a>[Qテーブルの初期化](#toc0_)

Qテーブルを初期化します。

学習を途中で中断した場合でも、このQテーブルの値を保存しておくことで、学習を途中から再開することができます。

In [5]:
q_table = np.zeros((len(States), len(Actions)))

# <a id='toc7_'></a>[Q学習のパラメータ設定](#toc0_)

Q学習におけるパラメータを設定します。

以下のパラメータを変更することで、学習の速度や性能が変わります。

In [ ]:
EPISODE_MAX = 1000 # 最大エピソード数
STEP_MAX = 4000 # 1エピソードあたりの最大ステップ数
LEARNING_RATE = 0.01 # 学習率
DISCOUNT_RATE = 0.9 # 割引率
EPSILON = 0.1 # ε-greedy法のε（ランダム行動を選択する確率）
LEARNING_COMPLETE_EPISODE_NUM = 4 # 学習が完了したと判断するエピソード数（今回の場合、移動ロボットがラインから外れずに「LEARNING_COMPLETE_EPISODE_NUM回連続で」一周出来た場合に学習完了とする）

# <a id='toc8_'></a>[Q学習の実行](#toc0_)

Q学習は、以下の手順で実行されます。

1. 初期状態$s_t$を取得
2. Qテーブルから、状態$s_t$において最もQ値が高い行動$a_t$を選択し、実行
    - ただし、一定の確率でランダムな行動を選択する（ε-greedy法）
3. 次の状態$s_{t+1}$を取得
4. 報酬$r_{t+1}$を取得
5. Q値$Q(s_t, a_t)$を更新し、Qテーブルに反映
6. 状態$s_t$を$s_{t+1}$に更新
7. 学習が完了するか、最大繰り返し回数に達するまで、2.〜6.を繰り返す

In [6]:
# ロボットを初期位置にセット
car_start_pos = [3.0, 0, 0.1]
car_start_orientation = pybullet.getQuaternionFromEuler([0.0, 0.0, -math.pi/2])
pybullet.resetBasePositionAndOrientation(car_id, car_start_pos, car_start_orientation)


# ボトムカメラの設定
projection_matrix = pybullet.computeProjectionMatrixFOV(fov=140.0,aspect=1.0,nearVal=0.04,farVal=100)
    
# リンクのインデックス
CAMERA_IDX = 8
CAMERA_TARGET_IDX = 9

# ジョイントのインデックス
RIGHT_WHEEL_JOINT_IDX = 0
LEFT_WHEEL_JOINT_IDX = 1

# 直進時の速度
BASE_SPEED = 30

# Q学習の初期化
q_learning = QLearning(q_table, LEARNING_RATE, DISCOUNT_RATE, EPSILON)
episode_complete_cnt = 0 # 移動ロボットがラインを外れずに「連続で」一周した回数

# 学習開始
for episode in range(EPISODE_MAX):

    # 学習が完了したら終了
    if episode_complete_cnt == LEARNING_COMPLETE_EPISODE_NUM:
        pybullet.removeAllUserDebugItems()
        pybullet.addUserDebugText(f"Learning is complete!!", [-2.3, 6.0, 0.1], textSize=2, textColorRGB=[1,0,0])
        pybullet.addUserDebugText(f"Episode: {episode}", [-0.7, 5.5, 0.1], textSize=1.5, textColorRGB=[1,0,0])
        pybullet.addUserDebugText(f"Q table", [-0.5, 5.0, 0.1], textSize=1.3, textColorRGB=[1,0,0])
        pybullet.addUserDebugText(f"| STATE \ ACTION | STRAIGHT | TURN LEFT | TURN RIGHT |", [-3.9, 4.6, 0.1], textSize=1.3, textColorRGB=[1,0,0])
        pybullet.addUserDebugText(f"----------------------------------------------------------------------------", [-3.9, 4.4, 0.1], textSize=1.3, textColorRGB=[1,0,0])
        pybullet.addUserDebugText(f"|   BLACK-BLACK   |    {q_table[0][0]:.2f}       |     {q_table[0][1]:.2f}        |      {q_table[0][2]:.2f}        |", [-3.9, 4.2, 0.1], textSize=1.3, textColorRGB=[1,0,0])
        pybullet.addUserDebugText(f"|   WHITE-BLACK   |    {q_table[1][0]:.2f}       |     {q_table[1][1]:.2f}        |      {q_table[1][2]:.2f}        |", [-3.9, 3.8, 0.1], textSize=1.3, textColorRGB=[1,0,0])
        pybullet.addUserDebugText(f"|   BLACK-WHITE   |    {q_table[2][0]:.2f}       |     {q_table[2][1]:.2f}        |      {q_table[2][2]:.2f}        |", [-3.9, 3.4, 0.1], textSize=1.3, textColorRGB=[1,0,0])
        pybullet.addUserDebugText(f"|   WHITE-WHITE   |    {q_table[3][0]:.2f}       |     {q_table[3][1]:.2f}        |      {q_table[3][2]:.2f}        |", [-3.9, 3.0, 0.1], textSize=1.3, textColorRGB=[1,0,0])
        break

    # 移動ロボットを初期位置、速度0にセット
    # エピソードごとに走行の向きを変える
    if episode % 2 == 0:
        car_start_pos = [3.0, 0, 0.1]
        car_yaw = -math.pi/2
    else:
        car_yaw = math.pi/2
        car_start_pos = [3.0, 0, 0.1]
    car_start_orientation = pybullet.getQuaternionFromEuler([0.0, 0.0, car_yaw])
    pybullet.resetBasePositionAndOrientation(car_id, car_start_pos, car_start_orientation)
    pybullet.setJointMotorControl2(car_id, RIGHT_WHEEL_JOINT_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=0)
    pybullet.setJointMotorControl2(car_id, LEFT_WHEEL_JOINT_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=0)

    # エピソードごとにQテーブルを表示
    # (実際は、ステップごとに毎回Qテーブルは更新されているが、addUserDebugTextは処理が重いため、エピソードごとに表示)
    pybullet.removeAllUserDebugItems()
    pybullet.addUserDebugText(f"Episode: {episode+1}", [-0.7, 5.5, 0.1], textSize=1.5, textColorRGB=[1,0,1])
    pybullet.addUserDebugText(f"Q table", [-0.5, 5.0, 0.1], textSize=1.3, textColorRGB=[1,0,1])
    pybullet.addUserDebugText(f"| STATE \ ACTION | STRAIGHT | TURN LEFT | TURN RIGHT |", [-3.9, 4.6, 0.1], textSize=1.3, textColorRGB=[1,0,1])
    pybullet.addUserDebugText(f"----------------------------------------------------------------------------", [-3.9, 4.4, 0.1], textSize=1.3, textColorRGB=[1,0,1])
    pybullet.addUserDebugText(f"|   BLACK-BLACK   |    {q_table[0][0]:.2f}       |     {q_table[0][1]:.2f}        |      {q_table[0][2]:.2f}        |", [-3.9, 4.2, 0.1], textSize=1.3, textColorRGB=[1,0,1])
    pybullet.addUserDebugText(f"|   WHITE-BLACK   |    {q_table[1][0]:.2f}       |     {q_table[1][1]:.2f}        |      {q_table[1][2]:.2f}        |", [-3.9, 3.8, 0.1], textSize=1.3, textColorRGB=[1,0,1])
    pybullet.addUserDebugText(f"|   BLACK-WHITE   |    {q_table[2][0]:.2f}       |     {q_table[2][1]:.2f}        |      {q_table[2][2]:.2f}        |", [-3.9, 3.4, 0.1], textSize=1.3, textColorRGB=[1,0,1])
    pybullet.addUserDebugText(f"|   WHITE-WHITE   |    {q_table[3][0]:.2f}       |     {q_table[3][1]:.2f}        |      {q_table[3][2]:.2f}        |", [-3.9, 3.0, 0.1], textSize=1.3, textColorRGB=[1,0,1])

    # 1. 初期状態を取得
    state = q_learning.get_state(car_id, CAMERA_IDX, CAMERA_TARGET_IDX, projection_matrix)
    
    # 1エピソードで最大STEP_MAX回の行動を取る
    for step in range(STEP_MAX):

        # 2. Qテーブルから行動を選択し、実行
        action = q_learning.get_action(state)
        if action == Actions.STRAIGHT:
            left_speed = BASE_SPEED
            right_speed = BASE_SPEED
        elif action == Actions.TURN_LEFT:
            left_speed = BASE_SPEED - 10
            right_speed = BASE_SPEED
        elif action == Actions.TURN_RIGHT:
            left_speed = BASE_SPEED
            right_speed = BASE_SPEED - 10
        pybullet.setJointMotorControl2(car_id, RIGHT_WHEEL_JOINT_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=right_speed)
        pybullet.setJointMotorControl2(car_id, LEFT_WHEEL_JOINT_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=left_speed)

        # 一時刻分、シミュレーションを進める
        pybullet.stepSimulation()
        time.sleep(1./240.)

        # 3. 次の状態を取得
        state_next = q_learning.get_state(car_id, CAMERA_IDX, CAMERA_TARGET_IDX, projection_matrix)

        # 4. 報酬を取得
        reward = q_learning.get_reward(state_next)

        # 5. Qテーブルの更新
        q_learning.update_q_table(state, action, state_next, reward)

        # 6. 状態を更新
        state = state_next

        # 移動ロボットがラインを外れたらエピソードを終了
        if state_next == States.WHITE_WHITE:
            episode_complete_cnt = 0
            break

        # 移動ロボットが、ラインを一周したら（＝スタート地点付近に戻ってきたら）エピソードを終了
        current_car_pos = pybullet.getBasePositionAndOrientation(car_id)[0]
        current_to_start_distance = (car_start_pos[0] - current_car_pos[0])**2 + (car_start_pos[1] - current_car_pos[1])**2
        if current_to_start_distance < 0.1**2 and step > 1000:
            episode_complete_cnt += 1
            break
        

KeyboardInterrupt: 

: 

# <a id='toc9_'></a>[おまけ：ライントレース用の画像生成](#toc0_)

以下コードを実行すると、指定の半径の円を描画した画像を生成します。

In [9]:
from PIL import Image, ImageDraw

dpi = 100

plane_width = 10 
plane_height = 10
line_radius = 3

image_width = (plane_width * dpi)
image_height = (plane_height * dpi)

image = Image.new('RGB', (image_width, image_height), (255, 255, 255))
draw = ImageDraw.Draw(image)

def gazebo_to_image_coords(x, y):
    x_image = int(((x + plane_width/2) / plane_width) * image_width)
    y_image = int((1 - (y + plane_height/2) / plane_height) * image_height)
    return x_image, y_image

center_x_gazebo, center_y_gazebo = 0, 0

center_x_image, center_y_image = gazebo_to_image_coords(center_x_gazebo, center_y_gazebo)
radius_image = int(line_radius * image_width / plane_width)

draw.ellipse((center_x_image - radius_image, center_y_image - radius_image, center_x_image + radius_image, center_y_image + radius_image), outline="black", width=15)

image.save('../texture/line_trace_ground.png', dpi=(dpi, dpi))

X connection to :0 broken (explicit kill or server shutdown).


: 